In [6]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
from tqdm import tqdm
from calculateColumnsForModel import calculateColumnsForModel, getMedianOfCollectedData, numberToRoleName

game_ids.xlsx랑 collected_data가 있어야 돌아감.

In [7]:
PARTICIPANT_NUMBER_OF_A_TEAM = 5
participant_ids_by_role = []
DETAIL_PATH = "../data/collected_data/"

In [8]:
def getstats(gameId, number):
    pass

In [9]:
strcolumn_dict = {"tournamentId":"str", "matchId":"str", "gameId":"str", "patch":"str", "esportsTeamId_Blue":"str", "esportsTeamId_Red":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
game_ids_df = pd.read_excel("../data/game_ids.xlsx", dtype=strcolumn_dict, index_col = 0)
#game_ids_df["startTime(match)"] = game_ids_df["startTime(match)"].apply(lambda x : datetime.datetime.strptime(x[:19], "%Y-%m-%dT%H:%M:%S"))
    # add_esportsTeamId_to_gameId.ipynb에서 이미 형식을 바꿔버려서 위 코드를 넣었을 때 에러가 났던 것. 코드가 커지니까 이제 슬슬 헷갈리네 이러다 실수하는데

game_ids_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8021 entries, 0 to 8022
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         8021 non-null   datetime64[ns]
 1   tournamentId             8021 non-null   object        
 2   blockName                8021 non-null   object        
 3   leagueName               8021 non-null   object        
 4   matchId                  8021 non-null   object        
 5   bestof                   8021 non-null   int64         
 6   blueteam_name            8021 non-null   object        
 7   blueteam_code            8021 non-null   object        
 8   blueteam_win             8021 non-null   int64         
 9   redteam_name             8021 non-null   object        
 10  redteam_code             8021 non-null   object        
 11  redteam_win              8021 non-null   int64         
 12  gameNumberInAMatch       8021 non-null 

In [10]:
#이거 넣어야 됨. id에 널값이 있는 게임은 누군지도 모르니까 걍 제외.
included_all_ids_df = game_ids_df[game_ids_df["esportsPlayerId_0"].notna() & game_ids_df["esportsPlayerId_1"].notna() & game_ids_df["esportsPlayerId_2"].notna() &\
                                game_ids_df["esportsPlayerId_3"].notna() & game_ids_df["esportsPlayerId_4"].notna() & game_ids_df["esportsPlayerId_5"].notna() &\
                                    game_ids_df["esportsPlayerId_6"].notna() & game_ids_df["esportsPlayerId_7"].notna() & game_ids_df["esportsPlayerId_8"].notna() &\
                                        game_ids_df["esportsPlayerId_9"].notna()]
included_all_ids_df.info(verbose=True, show_counts=True)
#결과 줄수 7949 나오면 맞음

<class 'pandas.core.frame.DataFrame'>
Index: 7934 entries, 0 to 8022
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         7934 non-null   datetime64[ns]
 1   tournamentId             7934 non-null   object        
 2   blockName                7934 non-null   object        
 3   leagueName               7934 non-null   object        
 4   matchId                  7934 non-null   object        
 5   bestof                   7934 non-null   int64         
 6   blueteam_name            7934 non-null   object        
 7   blueteam_code            7934 non-null   object        
 8   blueteam_win             7934 non-null   int64         
 9   redteam_name             7934 non-null   object        
 10  redteam_code             7934 non-null   object        
 11  redteam_win              7934 non-null   int64         
 12  gameNumberInAMatch       7934 non-null 

여기까지는 game_ids에 esportsteamId를 붙이는 코드였음. 집와서 gameId다시읽을때 str타입으로 읽는 코드 추가할 것. strcolumn_dict에 넣으면 됨.

In [11]:
included_all_ids_df_sorted = included_all_ids_df.sort_values(by=["startTime(match)", "gameNumberInAMatch"], ascending=[False, False]).reset_index() # 최근 경기가 위로 오도록 내림차순 정렬
included_all_ids_df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7934 entries, 0 to 7933
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   index                    7934 non-null   int64         
 1   startTime(match)         7934 non-null   datetime64[ns]
 2   tournamentId             7934 non-null   object        
 3   blockName                7934 non-null   object        
 4   leagueName               7934 non-null   object        
 5   matchId                  7934 non-null   object        
 6   bestof                   7934 non-null   int64         
 7   blueteam_name            7934 non-null   object        
 8   blueteam_code            7934 non-null   object        
 9   blueteam_win             7934 non-null   int64         
 10  redteam_name             7934 non-null   object        
 11  redteam_code             7934 non-null   object        
 12  redteam_win              7934 non-

In [12]:
included_all_ids_df_sorted.head(5)

,index,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,...,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side,esportsTeamId_Blue,esportsTeamId_Red
0,4020,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,...,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280,Blue,111691749922904280,107437032053655867
1,4019,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,...,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280,Blue,111691749922904280,107437032053655867
2,4018,2024-05-28 08:00:00,112439909522772195,Week 1,LCO,112439909523296494,2,Fury,FRY,0,...,108604888219092306,111758035988327173,100160799381721105,104668539132702977,101383792849832093,103643243250697054,101383792831678607,Red,111691745473532520,109675484017801229
3,4017,2024-05-28 08:00:00,112439909522772195,Week 1,LCO,112439909523296494,2,Fury,FRY,0,...,108604888219092306,111758035988327173,100160799381721105,104668539132702977,101383792849832093,103643243250697054,101383792831678607,Red,111691745473532520,109675484017801229
4,4016,2024-05-27 10:00:00,112439909522772195,Week 1,LCO,112439909523296515,2,Antic Esports,ANC,2,...,102206314367722668,105714276869190870,107634897325128022,107634941727734818,105709436604717285,99566406296347493,102546029192465275,Red,98767991921462763,111691747754390318


In [13]:

test_df = included_all_ids_df_sorted[(included_all_ids_df_sorted["esportsTeamId_Blue"] == "107437032053655867") | (included_all_ids_df_sorted["esportsTeamId_Red"] == "107437032053655867")]
test_df.head(11)

,index,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,...,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side,esportsTeamId_Blue,esportsTeamId_Red
0,4020,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,...,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280,Blue,111691749922904280,107437032053655867
1,4019,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,...,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280,Blue,111691749922904280,107437032053655867
720,4085,2024-02-28 07:00:00,111561353913765677,Knockouts,LCO,111691792353211336,3,Antic Esports,ANC,2,...,111720027901143523,111720029171950542,107634897325128022,99566406283268035,111720135678641636,98767975937423972,102546029192465275,Red,107437032053655867,111691747754390318
721,4084,2024-02-28 07:00:00,111561353913765677,Knockouts,LCO,111691792353211336,3,Antic Esports,ANC,2,...,98767975937423972,102546029192465275,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Blue,111691747754390318,107437032053655867
726,4083,2024-02-27 09:00:00,111561353913765677,Knockouts,LCO,111691792353211332,3,ION Global Esports,ION,0,...,111720048642120764,110456581491929979,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Red,111691749922904280,107437032053655867
727,4082,2024-02-27 09:00:00,111561353913765677,Knockouts,LCO,111691792353211332,3,ION Global Esports,ION,0,...,111720048642120764,110456581491929979,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Red,111691749922904280,107437032053655867
835,4072,2024-02-21 07:00:00,111561353913765677,Week 6,LCO,111691782055605145,2,Antic Esports,ANC,1,...,98767975937423972,102546029192465275,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Red,111691747754390318,107437032053655867
836,4071,2024-02-21 07:00:00,111561353913765677,Week 6,LCO,111691782055605145,2,Antic Esports,ANC,1,...,98767975937423972,102546029192465275,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Blue,111691747754390318,107437032053655867
847,4064,2024-02-19 07:00:00,111561353913765677,Week 6,LCO,111691782055605109,2,Dire Wolves,DW,0,...,102206314367722668,105714276869190870,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Red,98767991921462763,107437032053655867
848,4063,2024-02-19 07:00:00,111561353913765677,Week 6,LCO,111691782055605109,2,Dire Wolves,DW,0,...,102206314367722668,105714276869190870,110462937178590834,109758314971921663,107635918452357647,111720027901143523,111720029171950542,Red,98767991921462763,107437032053655867


In [14]:
test_df = included_all_ids_df_sorted[(included_all_ids_df_sorted["esportsPlayerId_0"] == "111720042622049047") | (included_all_ids_df_sorted["esportsPlayerId_5"] == "111720042622049047")]
test_df[["gameId", "esportsPlayerId_0", "esportsPlayerId_5"]].head(11)

,gameId,esportsPlayerId_0,esportsPlayerId_5
0,112439909523296571,111720042622049047,110462937178590834
1,112439909523296570,111720042622049047,110462937178590834
726,111691792353211334,111720042622049047,110462937178590834
727,111691792353211333,111720042622049047,110462937178590834
736,111691792353145780,100160799381721105,111720042622049047
737,111691792353145779,100160799381721105,111720042622049047
841,111691782055605156,105714271946959686,111720042622049047
842,111691782055605155,105714271946959686,111720042622049047
971,111691782055605150,107634897325128022,111720042622049047
972,111691782055605149,107634897325128022,111720042622049047


위 테스트 데이터에서 Team 111691749922904280 는 최근 10전 중 3승을 했다.

In [15]:
median_playerstat = getMedianOfCollectedData()    # 한번돌리는시간이 너무 오래결려서 테스트용 임시코드

100%|██████████| 8036/8036 [09:40<00:00, 13.84it/s]


In [27]:
# 나중에 현재기준 테이블 만들때는 row안에 데이터를 직접 넣고 가장 겉의 for문 하나 제거하고 뚝딱뚝딱 돌리면 되겠지 ㅎㅎ
# 대신 앞에 last_row_of 저건 읽어와야한다. 반복문에서 하나하나읽지않고 한번 읽고 끝내는 것이 스피드를 위해 핵심.

strcolumn_dict = {"gameId":"str", "esportsTeamId_Blue":"str", "esportsTeamId_Red":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
last_row_of_collected_datas_df = pd.read_excel("../data/last_row_of_collected_datas.xlsx", dtype = strcolumn_dict)

newbie = 0          # 평균 채워지는 사람 얼마나 되는지 테스트. 줄수 * 10중에
RANGE_OF_RECENT_GAME = 10 # 바꿔보는중
YEAR_DAYS = 365
HALF_OF_YEAR_DAYS = 183      # 2월 안껴있으면 대체로 맞을듯?
STAT_MEDIAN_MULTIPLIER = 0.7       # 출전경험이 없는 선수는 리그 중간값의 0.7을 곱한걸 넣어보자.(바꿔보는중)
PITCHERS_NUMBER_OF_A_PLAYER = 8         # 플레이어 스탯 갯수
final_time = included_all_ids_df_sorted.iloc[-1]["startTime(match)"]
pre_make_pivot_table_df = None
for idx, row in tqdm(included_all_ids_df_sorted.iterrows(), total = included_all_ids_df_sorted.shape[0]):
    #if idx + RANGE_OF_RECENT_GAME >= included_all_ids_df.shape[0]: # 거의 끝행에 도달하면 어차피 지난 데이터도 얼마 없기 때문에 스톱한다.
    if row["startTime(match)"] - final_time < timedelta(days=HALF_OF_YEAR_DAYS):  # 너무 오래전꺼는 과거의 데이터가 없어서 힘들다. 데이터시작부터 반년 이내 게임들은 아예 잘라버림
        break
    else:
        #print(f"현재 row : 블루팀 : {row["esportsTeamId_Blue"]}, 레드팀 : {row["esportsTeamId_Red"]}")
        previous_subdata_df = included_all_ids_df_sorted[idx+1:] # 현재 행의 바로 다음 행부터

        blueteam_wincount = 1       # 블루팀의 승리 횟수, 100%가 나오는걸 피하기 위해 임의로 1승을 추가한 것임.
        subrow_count = 0
        blueteam_golddiff_sum = 0
        blueteam_killdiff_sum = 0
        subdata_blueteam = previous_subdata_df[(previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Blue"]) | 
                                               (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Blue"])
                                               ]# 블루 레드팀 어디든 현재 블루팀과 팀이 같은 경기가 있을 떼
        for sub_idx, sub_row in subdata_blueteam.iterrows():
            if row["startTime(match)"] - sub_row["startTime(match)"] > timedelta(days=YEAR_DAYS): # 최근 1년 경기가 아니면(바꾸는중)
                break   
            else:
                subrow_count += 1
                target_game = last_row_of_collected_datas_df[last_row_of_collected_datas_df["gameId"] == sub_row["gameId"]].T.squeeze()
                if (sub_row["esportsTeamId_Blue"] == row["esportsTeamId_Blue"]):
                    blueteam_golddiff_sum += (target_game["blue_totalGold"] - target_game["red_totalGold"]) / target_game["duration"]
                    blueteam_killdiff_sum += (target_game["blue_totalKills"] - target_game["red_totalKills"]) / target_game["duration"]
                    if (sub_row["winner_side"] == "Blue"):
                        blueteam_wincount += 1
                elif (sub_row["esportsTeamId_Red"] == row["esportsTeamId_Blue"]):
                    blueteam_golddiff_sum += (target_game["red_totalGold"] - target_game["blue_totalGold"]) / target_game["duration"]
                    blueteam_killdiff_sum += (target_game["red_totalKills"] - target_game["blue_totalKills"]) / target_game["duration"]
                    if (sub_row["winner_side"] == "Red"):
                        blueteam_wincount += 1
         
        blueteam_winrate = blueteam_wincount / (subrow_count + 2)  # + 2의 의미는 임의로 2전을 추가해서 바로위에 wincount+1한거와 함께 1전 1패의 가상데이터를 넣어준거임.
        if subrow_count == 0:
            blueteam_golddiff = blueteam_golddiff_sum
            blueteam_killdiff = blueteam_killdiff_sum
        else:
            blueteam_golddiff = blueteam_golddiff_sum / subrow_count
            blueteam_killdiff = blueteam_killdiff_sum / subrow_count

        redteam_wincount = 1        # 레드팀의 승리 횟수, 100%가 나오는걸 피하기 위해 임의로 1승을 추가한 것임.
        subrow_count = 0
        redteam_golddiff_sum = 0
        redteam_killdiff_sum = 0
        subdata_redteam = previous_subdata_df[(previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Red"]) | 
                                              (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Red"])
                                              ]# 블루 레드팀 어디든 현재 레드팀과 팀이 같은 경기가 있을 떼
        for sub_idx, sub_row in subdata_redteam.iterrows():
            if row["startTime(match)"] - sub_row["startTime(match)"] > timedelta(days=YEAR_DAYS): # 최근 1년 경기가 아니면(바꾸는중)
                break
            else:
                subrow_count += 1
                target_game = last_row_of_collected_datas_df[last_row_of_collected_datas_df["gameId"] == sub_row["gameId"]].T.squeeze()
                if (sub_row["esportsTeamId_Blue"] == row["esportsTeamId_Red"]):
                    redteam_golddiff_sum += (target_game["blue_totalGold"] - target_game["red_totalGold"]) / target_game["duration"]
                    redteam_killdiff_sum += (target_game["blue_totalKills"] - target_game["red_totalKills"]) / target_game["duration"]
                    if (sub_row["winner_side"] == "Blue"):
                        redteam_wincount += 1
                elif (sub_row["esportsTeamId_Red"] == row["esportsTeamId_Red"]):
                    redteam_golddiff_sum += (target_game["red_totalGold"] - target_game["blue_totalGold"]) / target_game["duration"]
                    redteam_killdiff_sum += (target_game["red_totalKills"] - target_game["blue_totalKills"]) / target_game["duration"]
                    if (sub_row["winner_side"] == "Red"):
                        redteam_wincount += 1
        redteam_winrate = redteam_wincount / (subrow_count + 2)  # + 2의 의미는 임의로 2전을 추가해서 바로위에 wincount+1한거와 함께 1전 1패의 가상데이터를 넣어준거임. 이렇게 100%와 0%의 극단적인 경우를 피해봄
        if subrow_count == 0:
            redteam_golddiff = redteam_golddiff_sum
            redteam_killdiff = redteam_killdiff_sum
        else:
            redteam_golddiff = redteam_golddiff_sum / subrow_count
            redteam_killdiff = redteam_killdiff_sum / subrow_count
        
        headtohead_wincount = 1     # 같은 팀이 만났을때 블루팀!!의 승리 횟수 (나중에 rate를 구할 때 0~1의 값으로 나올 것임)
        subrow_count = 0
        headtohead_golddiff_sum = 0
        headtohead_killdiff_sum = 0
        subdata_headtohead = previous_subdata_df[
            ((previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Blue"]) & 
            (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Red"])) |
            ((previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Red"]) & 
            (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Blue"])) # 양쪽에 교차해서 들어가 있으면 전부
        ]
        for sub_idx, sub_row in subdata_headtohead.iterrows():
            if (subrow_count > RANGE_OF_RECENT_GAME) and (row["startTime(match)"] - sub_row["startTime(match)"] > timedelta(days=YEAR_DAYS)): # 10개 이상 상대전적 데이터가 모였고 최근 1년 경기가 아니면(바꿔보는중)
                break
            else:
                subrow_count += 1
                target_game = last_row_of_collected_datas_df[last_row_of_collected_datas_df["gameId"] == sub_row["gameId"]].T.squeeze()
                if (sub_row["esportsTeamId_Blue"] == row["esportsTeamId_Blue"]):
                    headtohead_golddiff_sum += (target_game["blue_totalGold"] - target_game["red_totalGold"]) / target_game["duration"]
                    headtohead_killdiff_sum += (target_game["blue_totalKills"] - target_game["red_totalKills"]) / target_game["duration"]
                    if (sub_row["winner_side"] == "Blue"): # 현재 블루팀과 비교할 블루팀이 같으면서 블루팀이 이겼을 경우
                        headtohead_wincount += 1
                elif (sub_row["esportsTeamId_Red"] == row["esportsTeamId_Blue"]):
                    headtohead_golddiff_sum += (target_game["red_totalGold"] - target_game["blue_totalGold"]) / target_game["duration"]
                    headtohead_killdiff_sum += (target_game["red_totalKills"] - target_game["blue_totalKills"]) / target_game["duration"]
                    if (sub_row["winner_side"] == "Red"): # 현재 블루팀과 레드팀이 비교할 게임에서 서로 진영을 바꾸었는데 레드팀이 이겼을 경우(결국 같은 팀이 이겼나를 보기 위한 조건이다)
                        headtohead_wincount += 1
        headtohead_winrate = headtohead_wincount / (subrow_count + 2)
        #print(f"팀 ID {row["esportsTeamId_Blue"]} 와 {row["esportsTeamId_Red"]} 이 붙었을 때 {row["esportsTeamId_Blue"]} 가 이긴 확률은 {headtohead_winrate * 100}% 입니다.")
        if subrow_count == 0:
            headtohead_golddiff = headtohead_golddiff_sum
            headtohead_killdiff = headtohead_killdiff_sum
        else:
            headtohead_golddiff = headtohead_golddiff_sum / subrow_count
            headtohead_killdiff = headtohead_killdiff_sum / subrow_count
        
        ##### 아래는 선수별 최근 n경기 스탯을 통해 폼을 뽑아내는 코드 #####
        desired_labels = [f'esportsPlayerId_{j}' for j in range(PARTICIPANT_NUMBER_OF_A_TEAM * 2)]
        for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 2):   # 선수 10명에 대해
            playerstat_list = []    # 선수별 최근 n경기의 기록을 저장할 곳
            role = i % 5
            subdata_playerstat = previous_subdata_df[(previous_subdata_df[f"esportsPlayerId_{role}"] == row[f"esportsPlayerId_{i}"]) |
                                                     (previous_subdata_df[f"esportsPlayerId_{role + PARTICIPANT_NUMBER_OF_A_TEAM}"] == row[f"esportsPlayerId_{i}"])].head(RANGE_OF_RECENT_GAME)
            number_of_past_games_of_player = subdata_playerstat.shape[0]
            #print(row[f"esportsPlayerId_{i}"])

            for sub_idx, sub_row in subdata_playerstat.iterrows():
                #딕셔너리로 만들어서 playerstat_list에 append하자. 그 다음에 형태를 보자.
                playerstat_dict_of_the_game = {}    # 한 경기에 대한 플레이어 활약상을 넣을 딕셔너리
                game_ids_of_sub_row = sub_row[desired_labels]
                matched_column = game_ids_of_sub_row[game_ids_of_sub_row.values == row[f"esportsPlayerId_{i}"]].index[0]      # ID가 서로 맞는 컬럼이 어디인지(블루팀인지레드팀인지 모르니까)
                matched_column_number = matched_column[matched_column.find("_")+1:]     # 번호 추출
                target_game = calculateColumnsForModel(last_row_of_collected_datas_df[last_row_of_collected_datas_df["gameId"] == sub_row["gameId"]].T.squeeze())
                #print(target_game["gameId"])
                playerstat_dict_of_the_game["kda"] = target_game[f"kda_{matched_column_number}"]
                playerstat_dict_of_the_game["killsPerTime"] = target_game[f"killsPerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["deathsPerTime"] = target_game[f"deathsPerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["assistsPerTime"] = target_game[f"assistsPerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["championDamageShare"] = target_game[f"championDamageShare_{matched_column_number}"]
                playerstat_dict_of_the_game["creepScorePerTime"] = target_game[f"creepScorePerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["wardsScorePerTime"] = target_game[f"wardsScorePerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["goldEarnedPerTime"] = target_game[f"goldEarnedPerTime_{matched_column_number}"]
                #print(playerstat_dict_of_the_game)
                playerstat_list.append(playerstat_dict_of_the_game)
                # 넣을거 : kda, championDamageShare, creepScore, wardsScore, goldEarned
            if number_of_past_games_of_player <= 1:
                newbie += 1
                playerstat_dict_of_the_game = {         # 빈 자리에 일반적인 평균을 채워 넣고
                    "kda" : median_playerstat[f"kdaof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "killsPerTime" : median_playerstat[f"killsPerTimeof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "deathsPerTime" : median_playerstat[f"deathsPerTimeof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "assistsPerTime" : median_playerstat[f"assistsPerTimeof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "championDamageShare" : median_playerstat[f"championDamageShareof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "creepScorePerTime" : median_playerstat[f"creepScorePerTimeof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "wardsScorePerTime" : median_playerstat[f"wardsScorePerTimeof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER,
                    "goldEarnedPerTime" : median_playerstat[f"goldEarnedPerTimeof{numberToRoleName(i)}"] * STAT_MEDIAN_MULTIPLIER
                }
            else:
                playerstat_keys = playerstat_list[0].keys()
                playerstat_values = {key: [] for key in playerstat_keys}
                for entry in playerstat_list:
                    for key in playerstat_keys:
                        playerstat_values[key].append(entry[key])
                playerstat_dict_of_the_game = {key : np.mean(playerstat_values[key]) for key in playerstat_keys}
            if i == 0:  # 첫번째선수니까 데이터프레임을 새로 만들고
                #print("playerstat_list : ", playerstat_list)
                player_form = pd.DataFrame(playerstat_dict_of_the_game, index=[0]).T
                player_form.reset_index(inplace = True)
                player_form.columns = ["elements", "formvalue"]

            else:       # 아니면 위아래로 이어붙이는 작업
                #print(player_form)
                temp_player_form = pd.DataFrame(playerstat_dict_of_the_game, index=[0]).T
                temp_player_form.reset_index(inplace=True)
                temp_player_form.columns = ["elements", "formvalue"]
                player_form = pd.concat([player_form, temp_player_form], ignore_index=True)
        #print(player_form)
        # 나온 선수별 폼을 팀별 승률과 이어붙이는 곳
        columns_of_role = []
        for i in range(2):  # 블루팀과 레드팀 2개
            columns_of_role.extend(["Top" for j in range(PITCHERS_NUMBER_OF_A_PLAYER)])
            columns_of_role.extend(["Jgl" for j in range(PITCHERS_NUMBER_OF_A_PLAYER)])
            columns_of_role.extend(["Mid" for j in range(PITCHERS_NUMBER_OF_A_PLAYER)])
            columns_of_role.extend(["Adc" for j in range(PITCHERS_NUMBER_OF_A_PLAYER)])
            columns_of_role.extend(["Spt" for j in range(PITCHERS_NUMBER_OF_A_PLAYER)])
        
        each_team_winrate_extended = [blueteam_winrate for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)] # 역할군 갯수가 5개, 총 25개만큼 복제 -> 5안에서 40개로 추가
        each_team_winrate_extended.extend([redteam_winrate for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)])
        each_team_golddiff_extended = [blueteam_golddiff for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)]
        each_team_golddiff_extended.extend([redteam_golddiff for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)])
        each_team_killdiff_extended = [blueteam_killdiff for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)]
        each_team_killdiff_extended.extend([redteam_killdiff for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)])
        columns_of_each_team_winrate_extended = ["Blue" for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)]
        columns_of_each_team_winrate_extended.extend(["Red" for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5)])
        headtohead_winrate_extended = [headtohead_winrate for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5 * 2)]
        headtohead_golddiff_extend = [headtohead_golddiff for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5 * 2)]
        headtohead_killdiff_extend = [headtohead_killdiff for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5 * 2)]
        winnerside_extended = [row["winner_side"] for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5 * 2)]
        gameId_extended = [row["gameId"] for i in range(PITCHERS_NUMBER_OF_A_PLAYER * 5 * 2)]
        
        each_team_winrate_dict = {                                  # 50개 행으로 만들 예정 -> 5안에서 80개로 추가. 왜냐하면 킬 뎃 어시를 각자 입력해보려고. kda로는 부족한듯?
            "gameId" : gameId_extended,
            "headtoHeadWinrate" : headtohead_winrate_extended,
            "headtoHeadGoldDiff" : headtohead_golddiff_extend,
            "headtoHeadKillDiff" : headtohead_killdiff_extend,
            "winner" : winnerside_extended,                         # 라벨, !!!! 제 일 중 요 !!!!
            "side" : columns_of_each_team_winrate_extended,         # "Blue" 또는 "Red", 피벗테이블의 컬럼으로 만들 예정
            "teamWinrate" : each_team_winrate_extended,             # Side에 따라 다른 값
            "teamGoldDiff" : each_team_golddiff_extended,
            "teamKillDiff" : each_team_killdiff_extended,
            "role" : columns_of_role
        }
        dataset_of_a_game_df = pd.DataFrame(each_team_winrate_dict)
        dataset_of_a_game_df = pd.concat([dataset_of_a_game_df, player_form], axis=1, ignore_index=True)
        dataset_of_a_game_df.columns = (["gameId", "headtoHeadWinrate", "headtoHeadGoldDiff", "headtoHeadKillDiff", "winner", "side", "teamWinrate", "teamGoldDiff", "teamKillDiff", "role", "elements", "formvalue"])
        #print(dataset_of_a_game_df)
        if pre_make_pivot_table_df is None:
            pre_make_pivot_table_df = dataset_of_a_game_df
        else:
            pre_make_pivot_table_df = pd.concat([pre_make_pivot_table_df, dataset_of_a_game_df], ignore_index=True)
        #print(pre_make_pivot_table_df)

print("newbie : ", newbie)
pre_make_pivot_table_df.to_excel("../data/pre_make_pivot_table_draft5.xlsx")    


 87%|████████▋ | 6884/7934 [10:30:31<1:36:10,  5.50s/it]


newbie :  2092


In [ ]:


fromvalue_pivot = pd.pivot_table(
    pre_make_pivot_table_df,
    values = "formvalue",
    index = ["gameId"],
    columns = ["side", "role", "elements"]
).reset_index()
fromvalue_pivot.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in fromvalue_pivot.columns]
fromvalue_pivot.rename(columns = {"gameId__" : "gameId"}, inplace = True)
teamwinrate_df = pre_make_pivot_table_df[["gameId", "side", "teamWinrate"]].drop_duplicates()

teamwinrate_df = teamwinrate_df.pivot(index='gameId', columns='side', values='teamWinrate')

# Subtract Red from Blue to get the new column
teamwinrate_df['teamWinrateDiff'] = teamwinrate_df['Blue'] - teamwinrate_df['Red']
teamwinrate_df.drop(["Blue", "Red"], axis=1, inplace=True)

# Reset index to turn gameId back into a column
teamwinrate_diff_df = teamwinrate_df.reset_index()
headtohead_and_winner_df = pre_make_pivot_table_df[["gameId", "headtoHeadWinrate", "winner"]].drop_duplicates()
final_pivot_table = fromvalue_pivot.merge(teamwinrate_diff_df, on = "gameId", how = "left")
final_pivot_table = final_pivot_table.merge(headtohead_and_winner_df, on = "gameId", how = "left")
final_pivot_table.head()


,gameId,Blue_Adc_championDamageShare,Blue_Adc_creepScorePerTime,Blue_Adc_goldEarnedPerTime,Blue_Adc_kda,Blue_Adc_wardsScorePerTime,Blue_Jgl_championDamageShare,Blue_Jgl_creepScorePerTime,Blue_Jgl_goldEarnedPerTime,Blue_Jgl_kda,...,Red_Spt_kda,Red_Spt_wardsScorePerTime,Red_Top_championDamageShare,Red_Top_creepScorePerTime,Red_Top_goldEarnedPerTime,Red_Top_kda,Red_Top_wardsScorePerTime,teamWinrateDiff,headtoHeadWinrate,winner
0,106269654661533486,0.292294,0.163173,6.974498,5.200,0.016950,0.145844,0.093639,5.574723,6.056667,...,5.520000,0.049224,0.255484,0.145089,6.799953,6.573333,0.015360,-0.098128,0.250000,Red
1,106269654661533487,0.282222,0.157684,6.691084,3.950,0.016688,0.147539,0.089636,5.469117,4.906667,...,5.786667,0.049284,0.267624,0.145650,6.922345,7.793333,0.015350,-0.119048,0.200000,Red
2,106269654661599026,0.283189,0.147146,7.363543,7.735,0.019430,0.157957,0.097901,5.713389,8.435000,...,5.582619,0.045983,0.272596,0.151100,7.034842,5.393333,0.015188,-0.066667,0.200000,Red
3,106269654661599027,0.259536,0.179423,7.427517,6.000,0.018119,0.174771,0.099356,5.900707,7.677143,...,6.220000,0.047948,0.273684,0.145948,6.929040,8.130000,0.013809,-0.086957,0.166667,Red
4,106269654661599028,0.277617,0.148115,7.255165,8.540,0.020133,0.169911,0.104458,6.019701,8.910000,...,2.632619,0.047078,0.297359,0.149443,7.136887,5.601667,0.015457,0.106383,0.857143,Blue


In [ ]:
final_pivot_table.to_excel("../data/dataset.xlsx")

In [ ]:
final_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7194 entries, 0 to 7193
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   gameId                        7194 non-null   object 
 1   Blue_Adc_championDamageShare  7194 non-null   float64
 2   Blue_Adc_creepScorePerTime    7194 non-null   float64
 3   Blue_Adc_goldEarnedPerTime    7194 non-null   float64
 4   Blue_Adc_kda                  7194 non-null   float64
 5   Blue_Adc_wardsScorePerTime    7194 non-null   float64
 6   Blue_Jgl_championDamageShare  7194 non-null   float64
 7   Blue_Jgl_creepScorePerTime    7194 non-null   float64
 8   Blue_Jgl_goldEarnedPerTime    7194 non-null   float64
 9   Blue_Jgl_kda                  7194 non-null   float64
 10  Blue_Jgl_wardsScorePerTime    7194 non-null   float64
 11  Blue_Mid_championDamageShare  7194 non-null   float64
 12  Blue_Mid_creepScorePerTime    7194 non-null   float64
 13  Blu

In [ ]:
final_pivot_table.describe()

,Blue_Adc_championDamageShare,Blue_Adc_creepScorePerTime,Blue_Adc_goldEarnedPerTime,Blue_Adc_kda,Blue_Adc_wardsScorePerTime,Blue_Jgl_championDamageShare,Blue_Jgl_creepScorePerTime,Blue_Jgl_goldEarnedPerTime,Blue_Jgl_kda,Blue_Jgl_wardsScorePerTime,...,Red_Spt_goldEarnedPerTime,Red_Spt_kda,Red_Spt_wardsScorePerTime,Red_Top_championDamageShare,Red_Top_creepScorePerTime,Red_Top_goldEarnedPerTime,Red_Top_kda,Red_Top_wardsScorePerTime,teamWinrateDiff,headtoHeadWinrate
count,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,...,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000,7194.000000
mean,0.276113,0.153950,7.299151,6.215429,0.016289,0.146274,0.091966,5.633765,5.523583,0.014861,...,4.086214,5.626504,0.042669,0.230872,0.134115,6.495914,4.525407,0.012511,0.002245,0.495038
std,0.032409,0.013381,0.531044,2.269533,0.003622,0.023871,0.008473,0.423589,2.188871,0.002938,...,0.326306,2.338421,0.008063,0.030368,0.009408,0.450459,1.798515,0.002395,0.202201,0.184054
min,0.168684,0.083170,5.203467,0.511111,0.001736,0.083050,0.065099,4.314235,0.569286,0.003540,...,3.260939,0.111111,0.004827,0.133625,0.071860,4.765270,0.166667,0.001490,-0.677419,0.083333
25%,0.255236,0.147298,6.965803,4.650417,0.014245,0.130388,0.086253,5.338454,3.968333,0.013018,...,3.876018,3.980893,0.039447,0.210632,0.128138,6.205970,3.280000,0.011297,-0.130073,0.333333
50%,0.275305,0.155048,7.286751,6.129330,0.016371,0.145013,0.091962,5.632477,5.484813,0.014856,...,4.059265,5.434881,0.043285,0.230866,0.133585,6.468180,4.427688,0.012669,0.000000,0.500000
75%,0.297032,0.162799,7.645657,7.652963,0.018121,0.159702,0.097246,5.897875,6.912500,0.016852,...,4.234219,7.019167,0.047787,0.249935,0.140147,6.770159,5.609226,0.014071,0.131168,0.642857
max,0.424150,0.188620,9.544249,17.466667,0.034457,0.271580,0.132075,8.276353,14.450000,0.024902,...,6.814723,16.250000,0.064336,0.351930,0.166117,8.710932,14.733333,0.026733,0.696970,0.923077


In [ ]:
final_pivot_table_without_gameid = final_pivot_table.drop("gameId", axis = 1).drop_duplicates()
final_pivot_table_without_gameid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7187 entries, 0 to 7193
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Blue_Adc_championDamageShare  7187 non-null   float64
 1   Blue_Adc_creepScorePerTime    7187 non-null   float64
 2   Blue_Adc_goldEarnedPerTime    7187 non-null   float64
 3   Blue_Adc_kda                  7187 non-null   float64
 4   Blue_Adc_wardsScorePerTime    7187 non-null   float64
 5   Blue_Jgl_championDamageShare  7187 non-null   float64
 6   Blue_Jgl_creepScorePerTime    7187 non-null   float64
 7   Blue_Jgl_goldEarnedPerTime    7187 non-null   float64
 8   Blue_Jgl_kda                  7187 non-null   float64
 9   Blue_Jgl_wardsScorePerTime    7187 non-null   float64
 10  Blue_Mid_championDamageShare  7187 non-null   float64
 11  Blue_Mid_creepScorePerTime    7187 non-null   float64
 12  Blue_Mid_goldEarnedPerTime    7187 non-null   float64
 13  Blue_Mid